# Python Crash Course_Part 2 Data Preparation
## Missing, Distinct Value, Scaling

## Full Day Workshop for user learn Data Science with Python
### 2017 Dec Timothy CL Lam
This is meant for internal usage, not for commercial purpose


In [86]:
# The code was removed by DSX for sharing.

,CHURN,Gender,Status,Children,Est Income,Car Owner,Age,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype,Usage,RatePlan
0,T,F,S,1.0,38000.00,N,24.393333,23.56,0.0,206.08,0.0,CC,Budget,Intnl_discount,229.64,3.0
1,F,M,M,2.0,29616.00,N,49.426667,29.78,0.0,45.50,0.0,CH,FreeLocal,Standard,75.29,2.0
2,F,M,M,0.0,19732.80,N,50.673333,24.81,0.0,22.44,0.0,CC,FreeLocal,Standard,47.25,3.0
3,F,M,S,2.0,96.33,N,56.473333,26.13,0.0,32.88,1.0,CC,Budget,Standard,59.01,1.0
4,F,F,M,2.0,52004.80,N,25.140000,5.03,0.0,23.11,0.0,CH,Budget,Intnl_discount,28.14,1.0


In [87]:
import pandas as pd
import numpy as np
from numpy import set_printoptions
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict, Counter

In [88]:
numerical_columns = list(df1.select_dtypes(include=[np.number]).columns)
categorical_columns = list(df1.select_dtypes(include=[object]).columns)
date_columns = list(df1.select_dtypes(include=['<M8[ns]']).columns)

In [89]:
categorical_columns[:]

['CHURN',
 'Gender',
 'Status',
 'Car Owner',
 'Paymethod',
 'LocalBilltype',
 'LongDistanceBilltype']

In [90]:
print "Loaded dataset"
print "   Rows: %s" % df1.shape[0]
print "   Columns: %s (%s num, %s cat, %s date)" % (df1.shape[1], 
                                                    len(numerical_columns), len(categorical_columns),
                                                    len(date_columns))

Loaded dataset
   Rows: 2066
   Columns: 16 (9 num, 7 cat, 0 date)


## Additional preparation
### Too Many Unique Values
Get rid of the columns that contain too many unique values

In [91]:
DROP_LIMIT_ABS = 200
CAT_DROP_LIMIT_RATIO = 0.5
for feature in categorical_columns:
    nu = df1[feature].nunique()
    
    if nu > DROP_LIMIT_ABS or nu > CAT_DROP_LIMIT_RATIO*df1.shape[0]:
        print "Dropping feature %s with %s values" % (feature, nu)
        columns_to_drop.append(feature)

### Missing Value Analysis
We then need to impute missing values

In [92]:
# Use mean for numerical features
for feature in numerical_columns:
    v = df1[feature].mean()
    if np.isnan(v):
        v = 0
    print "Filling %s with %s" % (feature, v)
    df1[feature] = df1[feature].fillna(v)
    
# Use mode for categorical features
for feature in categorical_columns:
    v = df1[feature].value_counts().index[0]
    df1[feature] = df1[feature].fillna(v)

Filling Children with 1.14617618587
Filling Est Income with 51514.0704647
Filling Age with 42.7839819342
Filling LongDistance with 16.1220764763
Filling International with 1.1911035818
Filling Local with 59.1580251694
Filling Dropped with 0.136011616651
Filling Usage with 75.907696031
Filling RatePlan with 2.51064859632


In [93]:
# Change sample size here
dataset_limit = 10000
# Toggle to keep dates in your dataset
keep_dates = False
# Toggle to dummify categorical features
dummify_categories = False



By default, date features are not kept. Modify the following cell to change that.

Also, by default, categorical columns are not included in the t-SNE data (preventing the creation of very sparse data). Modify the following cell to enable dummification.
We also use a categorical column for coloring if possible.

Keep the dates as features if requested by the user

In [94]:
columns_to_drop = []

if keep_dates:
    df[date_columns] = df[date_columns].astype(int)*1e-9
else:
    columns_to_drop.extend(date_columns)

## Dummy encoding
For all categorical features, we are going to "dummy-encode" them (also sometimes called one-hot encoding).

Basically, a categorical feature is replaced by one column per value. Each created value contains 0 or 1 depending on whether the original value was the one of the column.

In [95]:
# For categorical variables with more than that many values, we only keep the most frequent ones
LIMIT_DUMMIES = 100

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in features:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(df1, [x for x in categorical_columns if not x in columns_to_drop])


def dummy_encode_dataframe(df1):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, dummy_value.decode('utf-8'))
            df1[dummy_name] = (df1[feature] == dummy_value).astype(float)
        del df1[feature]
        print 'Dummy-encoded feature %s' % feature

dummy_encode_dataframe(df1)

Dummy-encoded feature Status
Dummy-encoded feature Paymethod
Dummy-encoded feature Gender
Dummy-encoded feature Car Owner
Dummy-encoded feature LongDistanceBilltype
Dummy-encoded feature CHURN
Dummy-encoded feature LocalBilltype


In [96]:
df1.head()

,Children,Est Income,Age,LongDistance,International,Local,Dropped,Usage,RatePlan,Status_value_M,...,Gender_value_F,Gender_value_M,Car Owner_value_N,Car Owner_value_Y,LongDistanceBilltype_value_Standard,LongDistanceBilltype_value_Intnl_discount,CHURN_value_F,CHURN_value_T,LocalBilltype_value_Budget,LocalBilltype_value_FreeLocal
0,1.0,38000.00,24.393333,23.56,0.0,206.08,0.0,229.64,3.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,2.0,29616.00,49.426667,29.78,0.0,45.50,0.0,75.29,2.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,19732.80,50.673333,24.81,0.0,22.44,0.0,47.25,3.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,2.0,96.33,56.473333,26.13,0.0,32.88,1.0,59.01,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,2.0,52004.80,25.140000,5.03,0.0,23.11,0.0,28.14,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


### There are number of Scalers. Please find the difference from here:
http://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html

### MinMaxScaler
Often this is referred toas normalization and attributes are often rescaled into the range between 0 and 1. This is
useful for optimization algorithms used in the core of machine learning algorithms like gradient
descent. It is also useful for algorithms that weight inputs like regression and neural networks
and algorithms that use distance measures like k-Nearest Neighbors. You can rescale your data
using scikit-learn using the MinMaxScaler class2.

In [102]:
X = df1.values
    
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler().fit(X)
X_std = mm.transform(X)
X_std.shape
X_std[:]

array([[ 0.5  ,  0.316,  0.187, ...,  1.   ,  1.   ,  0.   ],
       [ 1.   ,  0.246,  0.574, ...,  0.   ,  0.   ,  1.   ],
       [ 0.   ,  0.164,  0.593, ...,  0.   ,  0.   ,  1.   ],
       ..., 
       [ 0.   ,  0.699,  0.753, ...,  1.   ,  1.   ,  0.   ],
       [ 1.   ,  0.235,  0.409, ...,  1.   ,  0.   ,  1.   ],
       [ 0.   ,  0.238,  0.051, ...,  1.   ,  0.   ,  1.   ]])

### StandardScaler
Standardization is a useful technique to transform attributes with a Gaussian distribution and
diering means and standard deviations to a standard Gaussian distribution with a mean of
0 and a standard deviation of 1. It is most suitable for techniques that assume a Gaussian
distribution in the input variables and work better with rescaled data, such as linear regression,
logistic regression and linear discriminate analysis. You can standardize data using scikit-learn
with the StandardScaler class3.

In [103]:
Y = df1.values
    
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(Y)
Y_std = ss.transform(Y)
Y_std.shape
Y_std[:]


array([[-0.173, -0.439, -1.235, ...,  1.207,  0.914, -0.914],
       [ 1.013, -0.711,  0.446, ..., -0.829, -1.094,  1.094],
       [-1.36 , -1.032,  0.53 , ..., -0.829, -1.094,  1.094],
       ..., 
       [-1.36 ,  1.051,  1.225, ...,  1.207,  0.914, -0.914],
       [ 1.013, -0.756, -0.27 , ...,  1.207, -1.094,  1.094],
       [-1.36 , -0.744, -1.826, ...,  1.207, -1.094,  1.094]])

### RobustScaler
Unlike the previous scalers, the centering and scaling statistics of this scaler are based on percentiles and are therefore not influenced by a few number of very large marginal outliers. Consequently, the resulting range of the transformed feature values is larger than for the previous scalers and, more importantly, are approximately similar: for both features most of the transformed values lie in a [-2, 3] range as seen in the zoomed-in figure. Note that the outliers themselves are still present in the transformed data. If a separate outlier clipping is desirable, a non-linear transformation is required (see below).

In [107]:
Z = df1.values
    
from sklearn.preprocessing import RobustScaler
rs = RobustScaler(quantile_range=(25, 75)).fit(Z)
Z_std = rs.transform(Z)
Z_std.shape
Z_std[:]


array([[ 0.   , -0.313, -0.893, ...,  1.   ,  0.   ,  0.   ],
       [ 0.5  , -0.461,  0.165, ...,  0.   , -1.   ,  1.   ],
       [-0.5  , -0.634,  0.218, ...,  0.   , -1.   ,  1.   ],
       ..., 
       [-0.5  ,  0.492,  0.655, ...,  1.   ,  0.   ,  0.   ],
       [ 0.5  , -0.485, -0.286, ...,  1.   , -1.   ,  1.   ],
       [-0.5  , -0.479, -1.265, ...,  1.   , -1.   ,  1.   ]])

### QuantileTransformer
QuantileTransformer applies a non-linear transformation such that the probability density function of each feature will be mapped to a uniform distribution. In this case, all the data will be mapped in the range [0, 1], even the outliers which cannot be distinguished anymore from the inliers.
As RobustScaler, QuantileTransformer is robust to outliers in the sense that adding or removing outliers in the training set will yield approximately the same transformation on held out data. But contrary to RobustScaler, QuantileTransformer will also automatically collapse any outlier by setting them to the a priori defined range boundaries (0 and 1).

QuantileTransformer has an additional output_distribution parameter allowing to match a Gaussian distribution instead of a uniform distribution. Note that this non-parametetric transformer introduces saturation artifacts for extreme values.

In [108]:
Q = df1.values
    
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(output_distribution='normal').fit(Q)
Q_std = qt.transform(Q)
Q_std.shape
Q_std[:]


array([[-0.184, -0.351, -0.926, ...,  5.199,  5.199, -5.199],
       [ 5.199, -0.491,  0.303, ..., -5.199, -5.199,  5.199],
       [-5.199, -0.763,  0.363, ..., -5.199, -5.199,  5.199],
       ..., 
       [-5.199,  0.848,  1.245, ...,  5.199,  5.199, -5.199],
       [ 5.199, -0.56 , -0.432, ...,  5.199, -5.199,  5.199],
       [-5.199, -0.534, -1.733, ...,  5.199, -5.199,  5.199]])